# Homework 2 - Binary Mixing

For this homework, you will be doing two simulations to illustrate binary mixing of particles.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hoomd
import gsd.hoomd
import itertools

For this simulation, I have already set up the initial configuration for you, so all we have to do is initialize the simulation with our initial configuration gsd file. As always, we select the device we'd like to use for the simulation and create our simulation state. 

In [ ]:
cpu = hoomd.device.CPU()

sim = hoomd.Simulation(device=cpu, seed=1)
sim.create_state_from_gsd(filename='binary_config.gsd')
snap = sim.state.get_snapshot()

Now that we've initialized the simulation, we must create our integrator. 

In [ ]:
integrator = hoomd.md.Integrator(dt=0.005)
sim.operations.integrator = integrator

For this simulation, we are again going to use a Lennard-Jones pair potential to describe particle interactions. We have two different particle types in this system, so we must define interactions between particles of the same type as well as between particles of different types. Let's set that up now. 

In [ ]:
cell = hoomd.md.nlist.Cell(buffer=0.4)
lj = hoomd.md.pair.LJ(nlist=cell, default_r_cut=2.5)

lj.params[('typeA', 'typeA')] = dict(epsilon=1, sigma=1)
lj.params[('typeB', 'typeB')] = dict(epsilon=1, sigma=1)
lj.params[('typeA', 'typeB')] = dict(epsilon=1, sigma=1)

integrator.forces.append(lj)

And for the integrator, we are going to use an NVT integration method with the Nose-Hoover thermostat. This means that we are holding number of particles, volume, and temperature constant in our simulation while pressure and energy fluctuate. 

In [ ]:
nvt = hoomd.md.methods.ConstantVolume(filter=hoomd.filter.All(), 
                                      thermostat=hoomd.md.methods.thermostats.MTTK(kT=1.0, 
                                                                                tau=sim.operations.integrator.dt*100))
integrator.methods.append(nvt)

sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=1.0)

For this simulation, we want to model binary mixing which means we don't want the system to have periodic boundaries.  In HOOMD, we can create walls and place them on the faces of the simulation box to "turn-off" periodic boundaries. To do this, we must create a list of wall objects and then assign them all a force to dictate how the particles interact with the walls. 

Because we have a cubic simulation box, we want to create 6 walls, one for each face of the box. It's important to note here that HOOMD defines the center of the box to be the point (0, 0, 0). This means that the edges of the box fall at -L/2 and L/2, with L being the box length. 

To create a wall object, we define the geometry of the wall, which in our case is a Plane, and place it so that it intersects with a given point, or the `origin`. Then, we define a vector that is normal to the plane to correctly orient it in the space. I have created the first five walls already, so now it's your turn! **Fill in the origin point and normal vector that describes the bottom wall in the box (a wall on the xy plane).** For more information about walls in HOOMD, take a look at the [documentation](https://hoomd-blue.readthedocs.io/en/v4.1.0/module-hoomd-wall.html#hoomd.wall.Plane).

In [ ]:
#get the box length from the simulation snapshot
box_dim = snap.configuration.box
L = box_dim[0]

#set up the walls
boxwalls = [hoomd.wall.Plane(origin=(-L/2, 0, 0), normal=(1, 0, 0), open=False), #left well (-yz plane)
               hoomd.wall.Plane(origin=(L/2, 0, 0), normal=(-1, 0, 0), open=False), # right wall (yz plane)
               hoomd.wall.Plane(origin=(0, L/2, 0), normal=(0, -1, 0), open=False), #front wall (xz plane)
               hoomd.wall.Plane(origin=(0, -L/2, 0), normal=(0, 1, 0), open=False), #back wall (-xz plane)
               hoomd.wall.Plane(origin=(0, 0, L/2), normal=(0, 0, -1), open=False),#top wall (xy plane)
               hoomd.wall.Plane(origin=(), normal=(), open=False)] #bottom wall (-xy plane)
box_force = hoomd.md.external.wall.LJ(walls=boxwalls)
box_force.params[['typeA', 'typeB']] = dict(epsilon=1, sigma=1, r_cut=2**(1/6))
integrator.forces.append(box_force)

Because we want to see what happens when the particles mix, we should place a wall in between the two particle types at the start of the simulation. One of the important features of HOOMD walls is that only one side of the wall is "active", meaning only one side of the wall (the side that your normal vector points away from) will actually interact with the particles. Because of this, we must place two walls in the center of the box, one facing each side, to ensure that we have a partition. 

In [ ]:
partition = [hoomd.wall.Plane(origin=(0, 0, 0), normal=(0, 0, 1), open=False), 
               hoomd.wall.Plane(origin=(0, 0, 0), normal=(0, 0, -1), open=False)]
partition_force = hoomd.md.external.wall.LJ(walls=partition)
partition_force.params[['typeA','typeB']] = dict(epsilon=1, sigma=1, r_cut=2**(1/6))
integrator.forces.append(partition_force)

Now that we've set up the system, we can define our computes and writers so that we can log the data we get in this simulation. 

In [ ]:
thermo_properties = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())
sim.operations.computes.append(thermo_properties)

logger = hoomd.logging.Logger()
logger.add(thermo_properties)

gsd_writer = hoomd.write.GSD(filename='binary_mixing_walls_log.gsd', trigger = hoomd.trigger.Periodic(100), 
                             mode='wb', filter=hoomd.filter.All(), logger=logger)
sim.operations.writers.append(gsd_writer)

table_logger = hoomd.logging.Logger(categories=['scalar', 'string'])
table_logger.add(sim, quantities=['timestep', 'tps', 'walltime'])
table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=1000),
                          logger=table_logger)
sim.operations.writers.append(table)

Now we can run the initial steps of the simulation. 

In [ ]:
sim.run(10000)

Once we have allowed the system to equilibrate on each side of our partition, we can remove the partition and run the simulation again, allowing the particles to begin mixing. 

In [ ]:
integrator.forces.remove(partition_force)
sim.run(10000)
gsd_writer.flush()

Now that you have completed the simulation, **plot the potential energy of the system as a function of time.**

To better understand the impact of binary mixing and non-periodicity on our system, we should also take a look at a binary system without walls. 

**Create a simulation of binary mixing *without* walls using the same initial configuration. Compare the potential energy of the walled simulation to that of the non-walled simulation, and answer the following questions.** 
 * **Which system equilibrates faster? Why?**
 * **What differences do you notice in the energy fluctations between the two simulations? What could be the reason for these differences?**


I recommend you create a separate jupyter notebook for the second simulation to keep everything organized. For this assigment you should submit:
1. PDF versions of your jupyter notebook(s)
2. A PDF containing the plots (please label them and ensure the axes are the same scale) and answers to the above questions
3. A movie of each simulation, created with Ovito 

Note: It would be great if you could combine all of your pdfs into one pdf file, but if you forget/can't do that it's okay.